# <b>Лабораторна робота №3</b>
<b>з машинного навчання<br>
студента 3 курсу<br>
групи "Комп'ютерна математика"<br>
Демченка Андрія</b><br>

***Варіант №6***

Імпорт необхідних пакетів

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split as split_df

Підключаємо Гугл-Диск

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Завантажуємо датасет

In [3]:
headers = ('UserID', 'Game', 'Action', 'Hours', 'Not_Needed')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/steam-200k.csv', header=None, index_col=False, names=headers)

df.head()

,UserID,Game,Action,Hours,Not_Needed
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


Попередньо обробимо наші дані, заповнимо нулями ігри, які були куплені, але в них не грали

In [4]:
df['Hours_Played'] = df['Hours'].astype('float32')
df.loc[(df['Action'] == 'purchase') & (df['Hours'] == 1.0), 'Hours_Played'] = 0

df.head()

,UserID,Game,Action,Hours,Not_Needed,Hours_Played
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0,0.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0,273.0
2,151603712,Fallout 4,purchase,1.0,0,0.0
3,151603712,Fallout 4,play,87.0,0,87.0
4,151603712,Spore,purchase,1.0,0,0.0


Відсортуємо та відкинемо дублікати

In [5]:
df.UserID = df['UserID'].astype('int')
df = df.sort_values(['UserID', 'Game', 'Hours_Played'])

clean_df = df.drop_duplicates(['UserID', 'Game'], keep = 'last').drop(['Action', 'Hours', 'Not_Needed'], axis = 1)

data_matrix = clean_df.values

Покажімо результат

In [6]:
clean_df.head()

,UserID,Game,Hours_Played
65430,5250,Alien Swarm,4.9
65424,5250,Cities Skylines,144.0
65435,5250,Counter-Strike,0.0
65436,5250,Counter-Strike Source,0.0
65437,5250,Day of Defeat,0.0


Подивимось на кількість унікальних гравців та ігор

In [7]:
n_users = len(clean_df.UserID.unique())
n_games = len(clean_df.Game.unique())

print(f'Унікальних гравців: {n_users}')
print(f'Унікальних ігор: {n_games}')

Унікальних гравців: 12393
Унікальних ігор: 5155


Тепер визначимо між гравцями розподілені ігри

In [8]:
clean_df.Game.value_counts()

Dota 2                             4841
Team Fortress 2                    2323
Unturned                           1563
Counter-Strike Global Offensive    1412
Half-Life 2 Lost Coast              981
                                   ... 
RESCUE 2                              1
The Legend of Tango                   1
Foresight                             1
Dungeon of Elements                   1
Robot Rescue Revolution               1
Name: Game, Length: 5155, dtype: int64

Та навпаки ігри між гравцями

In [9]:
clean_df.UserID.value_counts()

62990992     1068
33865373      776
30246419      759
58345543      666
76892907      597
             ... 
52672853        1
170176187       1
163662283       1
201028044       1
217728407       1
Name: UserID, Length: 12393, dtype: int64

Оскільки ми матриця ігор-гравців розріджена, то ми визначимо скільки відсотків матриці буде заповнено

In [10]:
sparsity = clean_df.shape[0] / float(n_users * n_games)
print('{:.2%} буде заповнено'.format(sparsity * 100))

20.16% буде заповнено


Замінимо назви ігор та гравців на нові індекси

In [11]:
user2idx = {user: i for i, user in enumerate(clean_df.UserID.unique())}
idx2user = {i: user for user, i in user2idx.items()}

game2idx = {game: i for i, game in enumerate(clean_df.Game.unique())}
idx2game = {i: game for game, i in game2idx.items()}

user_idx = clean_df['UserID'].apply(lambda x: user2idx[x]).values
game_idx = clean_df['Game'].apply(lambda x: game2idx[x]).values

hours = clean_df['Hours_Played'].values

Перевіримо себе та створимо numpy матрицю з отриманих масивів

In [12]:
print(user_idx.shape, game_idx.shape, hours.shape)
data = np.hstack([user_idx.reshape(-1, 1), game_idx.reshape(-1, 1), hours.reshape(-1, 1)]).astype(np.int64)
data.shape

(128804,) (128804,) (128804,)


(128804, 3)

Тепер переходимо до створення класу для алгоритму PMF

In [13]:
class PMF(object):
    """Probabilistic Matrix Factorization algorithm.
    # Main variables
    :param n_features: Number of features
    :param epsilon: Learning rate value
    :param _lambda: L2 Regularization variable
    :param momentum: Value of the gradient momentum 
    :param max_epochs: Max epochs amount for algorithm processing 
    
    # SGD optimization
    :param n_batches: Batches number for each epoch
    :param batch_size: Amount of samples, used in one batch

    # Features vectors
    :param item_v: Vector for item feature
    :param user_v: Vector for user feature

    # RMSE samples
    :param rmse_train: RMSE train sample
    :param rmse_test: RMSE test sample
    """
    
    def __init__(self, n_features=10, epsilon=1, _lambda=0.1, momentum=0.8, max_epochs=20, n_batches=10, batch_size=1000):
        """Initialize the algorithm class and set parameters."""
        self.n_features = n_features
        self.epsilon = epsilon
        self._lambda = _lambda 
        self.momentum = momentum
        self.max_epochs = max_epochs
        
        self.n_batches = n_batches
        self.batch_size = batch_size

        self.item_v = None 
        self.user_v = None

        self.rmse_train = []
        self.rmse_test = []

    def fit(self, train_data, test_data):
        """Fit the model and compute RMSE.
        :param train_data: Training data sample
        :param test_data: Test data sample
        """
        # Find the mean of game played hours by players
        self.mean = np.mean(train_data[:, 2])  

        # Remember the lengths of train and test samples
        train_len = train_data.shape[0]
        test_len = test_data.shape[0]

        # Find maximal amount of users and games hours of both samples
        n_user = int(max(np.amax(train_data[:, 0]), np.amax(test_data[:, 0]))) + 1
        n_item = int(max(np.amax(train_data[:, 1]), np.amax(test_data[:, 1]))) + 1
        
        self.cur_epoch = 0  # Current epoch
        
        # Matrices of users and games hours normally distributed
        self.user_v = 0.1 * np.random.normal(0, .1, (n_user, self.n_features))  # N x D
        self.item_v = 0.1 * np.random.normal(0, .1, (n_item, self.n_features))  # M x D
        

        # Matrices of users and games for differences
        self.user_v_diff = np.zeros((n_user, self.n_features))  # N x D
        self.item_v_diff = np.zeros((n_item, self.n_features))  # M x D
            
        # Run Fitting while epoch won't reach given max amount
        while self.cur_epoch < self.max_epochs:
            self.cur_epoch += 1

            # Creating random order for training data 
            shuffled_order = np.arange(train_data.shape[0])
            np.random.shuffle(shuffled_order) 

            # Batch updating
            for batch in range(self.n_batches):
                cur_test = np.arange(self.batch_size * batch, self.batch_size * (batch + 1))
                batch_idx = np.mod(cur_test, shuffled_order.shape[0])

                # Get UserID and ItemID with computed batch id
                batch_UserID = np.array(train_data[shuffled_order[batch_idx], 0], dtype='int64')
                batch_ItemID = np.array(train_data[shuffled_order[batch_idx], 1], dtype='int64')

                # Compute actual output
                predict_out = np.sum(np.multiply(
                    self.user_v[batch_UserID, :],
                    self.item_v[batch_ItemID, :]),
                    axis=1
                )

                # Find errors with Loss Function
                err = predict_out - train_data[shuffled_order[batch_idx], 2] + self.mean

                # Compute gradients
                grads_user = 2 * np.multiply(err[:, np.newaxis], self.item_v[batch_ItemID, :]) \
                       + self._lambda * self.user_v[batch_UserID, :]
                grads_item = 2 * np.multiply(err[:, np.newaxis], self.user_v[batch_UserID, :]) \
                       + self._lambda * (self.item_v[batch_ItemID, :])

                # Prepare batch-sized gradiens
                grads_user_batched = np.zeros((n_user, self.n_features))
                grads_item_batched = np.zeros((n_item, self.n_features))

                # Fill batched gradients with needed values
                for i in range(self.batch_size):
                    grads_item_batched[batch_ItemID[i], :] += grads_item[i, :]
                    grads_user_batched[batch_UserID[i], :] += grads_user[i, :]

#               # Update Matrices with momentum function
                self.item_v_diff = self.momentum * self.item_v_diff + self.epsilon * grads_item_batched / self.batch_size
                self.user_v_diff = self.momentum * self.user_v_diff + self.epsilon * grads_user_batched / self.batch_size

                # Subtract differences and update matrices
                self.item_v = self.item_v - self.item_v_diff
                self.user_v = self.user_v - self.user_v_diff

                # Compute actual output after
                if batch == self.n_batches - 1:
                    predict_out = np.sum(np.multiply(
                        self.user_v[np.array(train_data[:, 0], dtype='int64'), :],
                        self.item_v[np.array(train_data[:, 1], dtype='int64'), :]
                    ), axis=1)  
                    

                    err = predict_out - train_data[:, 2] + self.mean
                    obj = np.linalg.norm(err) ** 2 \
                          + 0.5 * self._lambda * (np.linalg.norm(self.user_v) ** 2 + np.linalg.norm(self.item_v) ** 2)

                    self.rmse_train.append(np.sqrt(obj / train_len))

                # Compute validation error
                if batch == self.n_batches - 1:
                    predict_out = np.sum(np.multiply(self.user_v[np.array(test_data[:, 0], dtype='int64'), :],
                                                  self.item_v[np.array(test_data[:, 1], dtype='int64'), :]),
                                         axis=1)  # mean_inv subtracted
                    
                    # Find errors with Loss Function
                    err = predict_out - test_data[:, 2] + self.mean
                    self.rmse_test.append(np.linalg.norm(err) / np.sqrt(test_len))

                    # Output Log information
                    if batch == self.n_batches - 1:
                        print(f'Training data: {self.rmse_train[-1]}, Test data: {self.rmse_test[-1]}')

    def predict(self, id):
        """Predict method."""
        matrices_mult = np.dot(self.item_v, self.user_v[int(id), :])
        return matrices_mult + self.mean

    def get_accuracies(self, test_data, n=10):
        """Calculate and return mean of n best accuracies"""
        # Start accuracies of precision and recall
        precision_acc = 0.0
        recall_acc = 0.0

        # Get test users
        user_list = np.unique(test_data[:, 0])
        predictions, intersection_cnt = {}, {}

        # Fill None predictions indexes
        for usr in user_list:
            if predictions.get(usr, None) is None:
                predictions[usr] = np.argsort(self.predict(usr))[-n:]

        # Fill intersections
        for i in range(test_data.shape[0]):
            if test_data[i, 1] in predictions[test_data[i, 0]]:
                intersection_cnt[test_data[i, 0]] = intersection_cnt.get(test_data[i, 0], 0) + 1
        
        usr_pairs_cnt = np.bincount(np.array(test_data[:, 0], dtype='int64'))

        # Compute Precision and Recall Accuracies, finally
        for usr in user_list:
            precision_acc += intersection_cnt.get(usr, 0) / float(n)
            recall_acc += intersection_cnt.get(usr, 0) / float(usr_pairs_cnt[int(usr)])

        return precision_acc / len(user_list), recall_acc / len(user_list)

Розбиваємо дані на тренувальну та тестову частини

In [14]:
train, test = split_df(data, test_size=0.2, random_state=3)

Спочатку спробуємо виконати алгоритм для d = 10

In [15]:
%%time
pmf_1 = PMF(
    n_features=10,
    epsilon=0.05,
    _lambda=0.00000001,
    momentum=0.05,
    max_epochs=10,
    n_batches=80,
    batch_size=5000
)

pmf_1.fit(train, test)

Training data: 172.5246080340151, Test data: 166.8909065296319
Training data: 172.47341837445558, Test data: 166.8912604346441
Training data: 162.8145450793057, Test data: 166.8663052003893
Training data: 122.54187388037813, Test data: 166.52306472903422
Training data: 121.25398629726719, Test data: 166.48748596588314
Training data: 116.83859620530603, Test data: 167.8427992953371
Training data: 100.7059268614879, Test data: 171.16738045335305
Training data: 84.75795101750604, Test data: 174.48830001982788
Training data: 79.46058983616234, Test data: 176.19155701054305
Training data: 76.80594408584751, Test data: 176.32234676029745
CPU times: user 21.7 s, sys: 1.51 s, total: 23.2 s
Wall time: 22.1 s


Тепер підставимо d = 20

In [16]:
%%time
pmf_2 = PMF(
    n_features=20,
    epsilon=0.05,
    _lambda=0.00000001,
    momentum=0.05,
    max_epochs=10,
    n_batches=80,
    batch_size=5000
)

pmf_2.fit(train, test)

Training data: 172.52453222597046, Test data: 166.89086126859874
Training data: 172.4665550444155, Test data: 166.8906080717176
Training data: 161.4059138902015, Test data: 166.84818662561221
Training data: 122.4724869519535, Test data: 166.48539102420125
Training data: 120.8875760555901, Test data: 166.44530013342305
Training data: 114.0606869593529, Test data: 167.84290762127677
Training data: 94.25816528836836, Test data: 170.77690341041907
Training data: 81.46482675094072, Test data: 173.60195235625162
Training data: 77.46154355247013, Test data: 174.5007228016601
Training data: 75.5443534426611, Test data: 174.63674827188916
CPU times: user 24.1 s, sys: 1.77 s, total: 25.9 s
Wall time: 24.7 s


Помічаємо, що модель швидко перенавчається, тому візьмемо менше значення - d = 1

In [17]:
%%time
pmf_3 = PMF(
    n_features=1,
    epsilon=0.05,
    _lambda=0.00000001,
    momentum=0.05,
    max_epochs=10,
    n_batches=80,
    batch_size=5000
)

pmf_3.fit(train, test)

Training data: 172.5248825748737, Test data: 166.8908636226179
Training data: 172.52234330501992, Test data: 166.8908109479639
Training data: 172.06730121584758, Test data: 166.8884568703124
Training data: 130.91429675981345, Test data: 166.69268085391377
Training data: 122.2123975127902, Test data: 166.3312941477572
Training data: 120.7004588127307, Test data: 166.74197902905686
Training data: 117.56513500529896, Test data: 169.33332299602998
Training data: 112.56673836024206, Test data: 175.17607503181972
Training data: 103.83334367499471, Test data: 183.35218487678728
Training data: 99.16100163451165, Test data: 191.5196209233007
CPU times: user 20.3 s, sys: 1.02 s, total: 21.3 s
Wall time: 20.3 s


Вже краще! І врешті-решт спробуємо для більшого d (d = 5)

In [18]:
%%time
pmf_4 = PMF(
    n_features=5,
    epsilon=0.05,
    _lambda=0.00000001,
    momentum=0.05,
    max_epochs=10,
    n_batches=80,
    batch_size=5000
)

pmf_4.fit(train, test)

Training data: 172.52474030659116, Test data: 166.89087047041136
Training data: 172.49372737913927, Test data: 166.89089879210906
Training data: 166.78911251607434, Test data: 166.87564133411553
Training data: 122.64926403483975, Test data: 166.5424703767993
Training data: 121.68988167956064, Test data: 166.42908700230626
Training data: 119.3463846514913, Test data: 167.55015282938956
Training data: 112.71673327144251, Test data: 171.04294241675254
Training data: 95.48314157999468, Test data: 176.38403331787927
Training data: 83.50085186208852, Test data: 179.6100281056002
Training data: 79.4254635527228, Test data: 181.06771820254164
CPU times: user 20 s, sys: 1.12 s, total: 21.1 s
Wall time: 19.9 s


Помічаємо, що це найкращий з наших експериментів, тому залишаємо його. 
Та рахуємо для цієї ж моделі точність та повноту.

In [19]:
precision, recall = pmf_4.get_accuracies(test)
print(f'precision = {precision}, recall = {recall}')

precision = 0.03319105344032681, recall = 0.16698502121032485


Виходить, що результати моєї лабораторної роботи:

$ d = 5;$ <br> $\lambda = 0.033191$ <br> $\sigma^2=0.166985$ 
